<div style="color:#D81F26;
           display:fill;
           border-style: solid;
           border-color:#C1C1C1;
           font-size:14px;
           font-family:Calibri;
           background-color:#373737;">
<h2 style="text-align: center;
           padding: 10px;
           color:#FFFFFF;">
======= Keras MLP =======
</h2>
</div>

# About this notebook

This notebook is for submission to the Playground of Dec 2021. A simple Keras sequential neural network has been usedk. Different configuration of kernel initizlizers and activation functions have been experimented for performance comparison. No. of neurons and layers will be experimented later. 

Some codes are referred and copied from other notebooks.
​
## Summary
* Read the training and testing data sets with memory optimization  
* Perform EDA to examine the data and decide which features can be dropped
* Initialize  a 3-layer Keras sequential neural network with following model configuration
    * Kernel initializer of "lecun_normal" and "he_normal"
    * Activation function of "selu", "relu", "elu" and "swish"
* Get the optimal models for cross-validation and data submission
​

## Reference
* "TPS - Oct 2021 Model with Memory reduced" by S T MOHAMMED @stmohd (URL: https://www.kaggle.com/stmohd/tps-oct-2021-model-with-memory-reduced)
* "TPS - Nov 2021 - NN" by ŞAFAK TÜRKELI @sfktrkl (URL: [https://www.kaggle.com/sfktrkl/tps-nov-2021-nn](https://www.kaggle.com/sfktrkl/tps-nov-2021-nn))

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
# Basic library
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import accuracy_score
from scipy import stats

# Scaler
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Neural Network
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks

# Cross-Validation
from sklearn.model_selection import StratifiedKFold

# Memory optimization

In [3]:
df_train = pd.read_csv('./tabular-playground-series-dec-2021/train.csv')
df_test = pd.read_csv('./tabular-playground-series-dec-2021/test.csv')


## from: https://freedium.cfd/https://medium.com/towards-data-science/how-to-work-with-million-row-datasets-like-a-pro-76fb5c381cdd#:~:text=Today%2C%20I%20am%20here%20to%20share%20the%20concepts,natural%20as%20working%20with%20the%20Iris%20or%20Titanic.
def reduce_memory_usage(df, verbose=True):
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

df_train = reduce_memory_usage(df_train, verbose=True)
df_test = reduce_memory_usage(df_test, verbose=True)
print('Memory reduced')

Mem. usage decreased to 259.40 Mb (84.8% reduction)
Mem. usage decreased to 63.90 Mb (84.8% reduction)
Memory reduced


In [4]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000000 entries, 0 to 3999999
Data columns (total 56 columns):
 #   Column                              Dtype
---  ------                              -----
 0   Id                                  int32
 1   Elevation                           int16
 2   Aspect                              int16
 3   Slope                               int8 
 4   Horizontal_Distance_To_Hydrology    int16
 5   Vertical_Distance_To_Hydrology      int16
 6   Horizontal_Distance_To_Roadways     int16
 7   Hillshade_9am                       int16
 8   Hillshade_Noon                      int16
 9   Hillshade_3pm                       int16
 10  Horizontal_Distance_To_Fire_Points  int16
 11  Wilderness_Area1                    int8 
 12  Wilderness_Area2                    int8 
 13  Wilderness_Area3                    int8 
 14  Wilderness_Area4                    int8 
 15  Soil_Type1                          int8 
 16  Soil_Type2                          

In [5]:
#df_train.hist(bins=25, figsize = (20,20))

# Statistics of each variable

In [6]:
df_train.iloc[:,6:20].describe()

,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area1,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4,Soil_Type1,Soil_Type2,Soil_Type3,Soil_Type4,Soil_Type5
count,4.000000e+06,4.000000e+06,4.000000e+06,4.000000e+06,4.000000e+06,4.000000e+06,4.000000e+06,4.000000e+06,4.000000e+06,4.000000e+06,4.000000e+06,4.000000e+06,4.000000e+06,4.000000e+06
mean,1.766642e+03,2.118375e+02,2.210614e+02,1.408109e+02,1.581407e+03,2.611930e-01,4.166100e-02,6.535732e-01,2.181900e-02,1.684150e-02,3.089600e-02,4.275500e-03,3.791275e-02,1.571525e-02
std,1.315610e+03,3.075996e+01,2.223134e+01,4.369864e+01,1.127616e+03,4.392849e-01,1.998133e-01,4.758312e-01,1.460922e-01,1.286774e-01,1.730360e-01,6.524738e-02,1.909853e-01,1.243716e-01
min,-2.870000e+02,-4.000000e+00,4.900000e+01,-5.300000e+01,-2.770000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.220000e+02,1.980000e+02,2.100000e+02,1.150000e+02,7.810000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.436000e+03,2.180000e+02,2.240000e+02,1.420000e+02,1.361000e+03,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,2.365000e+03,2.330000e+02,2.370000e+02,1.690000e+02,2.084000e+03,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,7.666000e+03,3.010000e+02,2.790000e+02,2.720000e+02,8.075000e+03,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


#### From the describe() functions above, the variable Soil_Type7 and Soil_Type15 can be removed due to same value for all instances

# Distribution of Target

In [7]:
df_train['Cover_Type'].value_counts()

Cover_Type
2    2262087
1    1468136
3     195712
7      62261
6      11426
4        377
5          1
Name: count, dtype: int64

### From the value_counts() function above, there is only 1 instance for class 5.  Thus, treat it as outlier and remove it from the dataset

In [8]:
#df_train = df_train[df_train['Cover_Type'] != 5]

# Preparation of model data

In [9]:
# Get train data without the target and ids
X = df_train.iloc[:, 1:-1].copy()
X.drop(columns=['Soil_Type7', 'Soil_Type15'], inplace=True)

# Get the target
le = LabelEncoder()
y = le.fit_transform(df_train['Cover_Type'])

# Create test X, drop ids.
test_X = df_test.iloc[:, 1:].copy()
test_X.drop(columns=['Soil_Type7', 'Soil_Type15'], inplace=True)

In [10]:
del df_train, df_test

## Training History

The following configuration had been tried.

    # layers.Dense(64, activation='selu')      
    # layers.Dense(64, kernel_initializer='lecun_normal', activation='selu')    
    # layers.Dense(64, kernel_initializer='he_normal', activation='relu')      
    # layers.Dense(64, kernel_initializer='he_normal', activation='elu')
    # layers.Dense(64, activation='swish')

```
Configuration 1: 
    callbacks: EarlyStopping min_delta=0.001, patience=20
    EPOCHS: 50, BATCH_SIZE: 512, N_SPLITS: 5
    150 neurons, activation='selu', BatchNormalization
    70 neurons, activation='selu', BatchNormalization
    40 neurons, activation='selu', BatchNormalization
    7 neurons, activation='softmax'
    
    Overall Mean AUC:  0.9608532499999999
```
```
Configuration 2: 
    callbacks: EarlyStopping min_delta=0.001, patience=20
    EPOCHS: 50, BATCH_SIZE: 512, N_SPLITS: 5
    150 neurons, activation='selu', kernel_initializer='lecun_normal', BatchNormalization
    70 neurons, activation='selu', kernel_initializer='lecun_normal', BatchNormalization
    40 neurons, activation='selu', kernel_initializer='lecun_normal', BatchNormalization
    7 neurons, activation='softmax'
    
    Overall Mean AUC:  0.96171275
```
```
Configuration 3: 
    callbacks: EarlyStopping min_delta=0.001, patience=20
    EPOCHS: 50, BATCH_SIZE: 512, N_SPLITS: 5
    150 neurons, activation='relu', kernel_initializer='he_normal', BatchNormalization
    70 neurons, activation='relu', kernel_initializer='he_normal', BatchNormalization
    40 neurons, activation='relu', kernel_initializer='he_normal', BatchNormalization
    7 neurons, activation='softmax'
    
    Overall Mean AUC: 0.9615895
```
```
Configuration 4: 
    callbacks: EarlyStopping min_delta=0.001, patience=20
    EPOCHS: 50, BATCH_SIZE: 512, N_SPLITS: 5
    150 neurons, activation='swish', BatchNormalization
    70 neurons, activation='swish', BatchNormalization
    40 neurons, activation='swish', BatchNormalization
    7 neurons, activation='softmax'
    
    Overall Mean AUC: 0.9612097500000001
```    
```
Configuration 5: 
    callbacks: EarlyStopping min_delta=0.001, patience=20
    EPOCHS: 50, BATCH_SIZE: 512, N_SPLITS: 5
    150 neurons, activation='elu', kernel_initializer='he_normal', BatchNormalization
    70 neurons, activation='elu', kernel_initializer='he_normal', BatchNormalization
    40 neurons, activation='elu', kernel_initializer='he_normal', BatchNormalization
    7 neurons, activation='softmax'
    
    Overall Mean AUC: 0.96141325
```

---



# Model -  Keras NN

In [11]:
# https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping
early_stopping = callbacks.EarlyStopping(
    monitor="val_acc", 
    min_delta=0.001,           # Minimium amount of change to count as an improvement
    patience=5,               # How many epochs to wait before stopping
    restore_best_weights=True)

In [12]:
# https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ReduceLROnPlateau
reduce_lr = callbacks.ReduceLROnPlateau(
    monitor='val_loss', 
# Factor by which the learning rate will be reduced        
#    factor=0.1,                
    factor=0.2,                # Factor by which the learning rate will be reduced
    patience=5,                # Number of epochs with no improvement
    min_lr=0.001)              # Lower bound on the learning rate

In [13]:
# CALLBACKS = [early_stopping]

In [14]:
# Play with those configurations...
EPOCHS = 100
BATCH_SIZE = 4096
N_SPLITS = 20

In [ ]:
# w.X+b = y
# y = mx+c

# w = w - alpha.delta(w)

In [15]:
# model = keras.Sequential()
# model.add(tf.keras.layers.Dense(600, kernel_initializer='lecun_normal', activation='selu'))
# model.add(tf.keras.layers.Dense(300, kernel_initializer='lecun_normal', activation='selu'))
# model.add(tf.keras.layers.Dense(150, kernel_initializer='lecun_normal', activation='selu'))
# model.add(tf.keras.layers.Dense(75, kernel_initializer='lecun_normal', activation='selu'))
# model.add(tf.keras.layers.Dense(7, activation='softmax'))

model = keras.Sequential([
    layers.Dense(600,  activation='selu', kernel_initializer='lecun_normal', input_shape=[X.shape[1]]),
    layers.BatchNormalization(),
    layers.Dense(300,  activation='selu', kernel_initializer='lecun_normal'),   
    layers.BatchNormalization(),
    layers.Dense(150,  activation='selu', kernel_initializer='lecun_normal'),    
    layers.BatchNormalization(),
    layers.Dense(75,  activation='selu', kernel_initializer='lecun_normal'),    
    layers.BatchNormalization(),    
    # For a binary classification function use sigmoid
    layers.Dense(7, activation='softmax')])

C:\Users\sadiq\anaconda3\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [16]:
model.compile(optimizer="adam", loss='sparse_categorical_crossentropy', metrics=['acc'])

In [17]:
# fold = 0
#test_predictions = np.zeros(test_X.shape[0])
skf = StratifiedKFold(n_splits=N_SPLITS, random_state=48, shuffle=True)
#scores = {fold:None for fold in range(skf.n_splits)}
model_list = []
accuracy_list  = []
fold_list  = []
test_predictions = []

for fold,(train_idx, test_idx) in enumerate(skf.split(X, y)):
# for train_idx, test_idx in skf.split(X, y):
    train_X, val_X = X.iloc[train_idx], X.iloc[test_idx]
    train_y, val_y = y[train_idx], y[test_idx]

    # Train
    preproc = StandardScaler() # I tried QuantileTransformer, but StandardScaler seems to be better by 0.005
    train_X = preproc.fit_transform(train_X)
    val_X = preproc.transform(val_X)
    
    # Test
    pred_X = preproc.transform(test_X)

    history = model.fit(
        train_X, train_y,
        validation_data=(val_X, val_y), 
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        callbacks=[reduce_lr, early_stopping],        # Put your callbacks in a list
        verbose=0)                  # Turn off training log

    val_y_pred = model.predict(val_X, batch_size=BATCH_SIZE)
    val_y_pred = np.argmax(val_y_pred, axis=1)

    # Evaluation
    accuracy = accuracy_score(val_y, val_y_pred)
    
    accuracy_list.append(accuracy)
    model_list.append(history)
    fold_list.append(fold)
    
    print('Kfold = {}, Acc = {:0.5f}'.format(fold, accuracy))
    
    
    
    pred_y = model.predict(pred_X, batch_size=BATCH_SIZE)
    pred_y = np.argmax(pred_y, axis=1)
    test_predictions.append(pred_y)
    
    
#overall_auc = [np.max(scores[fold]['val_acc']) for fold in range(skf.n_splits)]
print('Overall Mean AUC: ', np.mean(accuracy_list))

C:\Users\sadiq\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=20.
  warnings.warn(


49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step
Kfold = 0, Acc = 0.96073
245/245 ━━━━━━━━━━━━━━━━━━━━ 8s 33ms/step
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step
Kfold = 1, Acc = 0.96066
245/245 ━━━━━━━━━━━━━━━━━━━━ 8s 33ms/step
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step
Kfold = 2, Acc = 0.96125
245/245 ━━━━━━━━━━━━━━━━━━━━ 7s 28ms/step
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step
Kfold = 3, Acc = 0.96236
245/245 ━━━━━━━━━━━━━━━━━━━━ 9s 36ms/step
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step
Kfold = 4, Acc = 0.96164
245/245 ━━━━━━━━━━━━━━━━━━━━ 8s 32ms/step
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step
Kfold = 5, Acc = 0.96225
245/245 ━━━━━━━━━━━━━━━━━━━━ 8s 31ms/step
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step
Kfold = 6, Acc = 0.96296
245/245 ━━━━━━━━━━━━━━━━━━━━ 7s 28ms/step
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step
Kfold = 7, Acc = 0.96261
245/245 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step
Kfold = 8, Acc = 0.96311
245/245 ━━━━━━━━━━━━━━━━━━━━ 8s 31ms/step
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/st